# In class exercise 10

The purpose of the exercise is to practice different machine learning algorithms for text clustering
Please downlad the dataset by using the following link.  https://www.kaggle.com/PromptCloudHQ/amazon-reviews-unlocked-mobile-phones
(You can also use different text data which you want)

Apply the listed clustering methods to the dataset:

K means, 
DBSCAN,
Hierarchical clustering. 

You can refer to of the codes from  the follwing link below. 
https://www.kaggle.com/karthik3890/text-clustering 
    

In [75]:
import re
import nltk
import random

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')

In [76]:
def review_to_wordlist( review, remove_stopwords=True):
    
    review_text = BeautifulSoup(review).get_text()

    review_text = re.sub("[^a-zA-Z]"," ", review)
   
    words = review_text.lower().split()
    
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]

    b=[]
    stemmer = english_stemmer #PorterStemmer()
    for word in words:
        b.append(stemmer.stem(word))

    
    return(b)

In [77]:
data_file = 'Amazon_Unlocked_Mobile.csv'

n = 413000  
s = 20000 
skip = sorted(random.sample(range(1,n),n-s))


data = pd.read_csv( data_file, delimiter = ",", skiprows = skip)

In [78]:
data.shape

(20840, 6)

In [79]:
data = data[data['Reviews'].isnull()==False]

In [82]:
clean_data_reviews = []
for review in data['Reviews']:
    clean_data_reviews.append( " ".join(review_to_wordlist(review)))

# K-Means

In [112]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(clean_data_reviews)

true_k = 6
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print

print("\n")
print("Prediction")
Y = vectorizer.transform(["the phone is great."])
prediction = model.predict(Y)
print(prediction)

Y = vectorizer.transform(["the phone is not working."])
prediction = model.predict(Y)
print(prediction)

Top terms per cluster:
Cluster 0:
 phone
 work
 love
 use
 like
 great
 good
 nice
 batteri
 new
Cluster 1:
 product
 good
 great
 excel
 recommend
 thank
 nice
 seller
 expect
 ship
Cluster 2:
 great
 phone
 work
 price
 love
 buy
 condit
 deal
 problem
 far
Cluster 3:
 excel
 product
 phone
 thank
 condit
 love
 good
 recommend
 seller
 smartphon
Cluster 4:
 good
 excelent
 phone
 price
 thank
 work
 far
 qualiti
 buy
 like
Cluster 5:
 perfect
 work
 phone
 condit
 thank
 everyth
 love
 great
 new
 good


Prediction
[2]
[0]


# DBSCAN

In [122]:
from sklearn.cluster import DBSCAN

In [124]:
db1 = DBSCAN(eps=0.9, min_samples=3).fit(X)
core_samples_mask = np.zeros_like(db1.labels_, dtype=bool)
core_samples_mask[db1.core_sample_indices_] = True
labels1 = db1.labels_
n_clusters_ = len(set(labels1)) - (1 if -1 in labels1 else 0) # Number of clusters in labels
print(labels1)
print()

clusters1 = {}
for c, i in enumerate(labels1):
    if i == -1:
        continue
    elif i in clusters1:
        clusters1[i].append( clean_data_reviews[c] )
    else:
        clusters1[i] = [clean_data_reviews[c]]
for c in clusters1:
    print(clusters1[c])
    print()

[-1 -1 -1 ... -1  1 -1]

['batteri die less hour', 'batteri last two hour', 'batteri last hour', 'star phone die fast', 'phone week work good batteri die fast', 'phone expect batteri die realli quick though problem phone', 'batteri last two hour', 'phone crap batteri hard last hour phone die', 'phone die week', 'batteri die fast', 'die complet month', 'work fine exact look problem batteri die pretti fast', 'phone nice charger die within month', 'live batteri hour', 'phone batteri die fast camera would load', 'batteri die month use receiv phone die disappoint product', 'batteri die hour', 'phone great batteri die realli fast']

['internet', 'work nice price bought mani', 'seem work fine', 'excel', 'phone turn batteri dead charg', 'excel thank', 'ok', 'excel phone function recommend', 'gracia', 'excel product', 'featur work', 'venezuela take g movilnet', 'excel', 'excelent', 'great phone', 'excel', 'okay', 'love', 'featur work', 'recomendado', 'great phone good valu money', 'product arri

# Hierarchical Clustering

In [ ]:
vectorizer = CountVectorizer(stop_words='english') 
dtm2 = vectorizer.fit_transform(clean_data_reviews) 
vocab2 = vectorizer.get_feature_names()
dtm2 = dtm2.toarray()


# Plot dendrogram 
cos_dist2 = 1 - cosine_similarity(dtm2)
linkage_matrix2 = ward(cos_dist2)
dendrogram(linkage_matrix2, color_threshold=0.6*max(linkage_matrix2[:,2]), orientation="right", labels=names2)
plt.tight_layout()
plt.show()

# In one paragraph, please compare K means, DBSCAN and Hierarchical clustering. 

Conlusion

As seen above the clustering models, they presented different results. Surely, they do have some advantages and disadvatages, howvver, the K means is the most flexible one as it is easy to interpret. DBSCAN may not be realay effective, as seen above, it is not able to inclunce the noise and handle with large dataset. Hierarchial clustering has also same disadvatages with DBSCAN. At this point, it is more suitable to use K-means by applying some methods to select K optimal values. 